# Testing

In [1]:
import sys
import pandas as pd
import yahooquery as yq
from tqdm import tqdm

In [2]:
sys.path.insert(0, '../')
import os
import pandas as pd
import model.investor_views as views
from model.model import Model
from api.api import DataAPI
import utils.vif as vif

## Model

In [3]:
num_years=100 # years
bounds=(-0.3, 1.0) # percent
gamma=0.0 # decimal
min_weight=0.05 # percent
margin_rate=0.132 # percent
long_weight=1.3 # percent
short_weight=0.3 # percent
frequency=252 # periods per year
period='max' # '1d', '5d', '7d', '60d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max'
api_source='bloomberg' # yahoo or bloomberg
market_symbol='SPY' # benchmark index ticker
etf_path='C:/Code/Python/pycape/data/etf_cape_return_forecast.csv'
stock_path='C:/Code/Python/pycape/data/stock_cape_return_forecast.csv'

In [4]:
def get_cape_forecast(filepath, min_long_return=0.15, max_short_return=0.0):
    """
    :description: Get equity ETF data and filter ETFs by expected return

    :param filepath: Filepath to cape forecasts
    :type filepath: str, required
    :param min_long_return: Minimum required long return, default is 0.15
    :type min_long_return: float, optional
    :param max_short_return: Maximum tolerable short return, default is 0.0
    :type max_short_return: float, optional
    :return: Selected equity ETFs
    :rtype: pd.DataFrame
    """
    equity_etf_csv = pd.read_csv(r'{}'.format(filepath))

    df = equity_etf_csv[
        (equity_etf_csv['FWD_RETURN_FORECAST'] >= min_long_return) |
        (equity_etf_csv['FWD_RETURN_FORECAST'] <= max_short_return)
        ]

    return df

In [5]:
selected_equity_etfs = get_cape_forecast(etf_path)
selected_equity_etfs

,TICKER,NAME,INDEX_NAME,DESCRIPTION,CAPE,CAPE_ZSCORE,LN_CAPE_ZSCORE,FWD_RETURN_FORECAST,LOWER_CONFIDENCE,UPPER_CONFIDENCE,F_PVALUE,MINIMUM_CAPE,MAXIMUM_CAPE,RSQUARED,INDEX_TICKER
0,PSCD,Invesco S&P SmallCap Consumer Discretionary ETF,S&P 600 Consumer Discretionary Sector GICS Lev...,Equity Consumer Discretionary,13.26,-1.44,-1.49,0.2132,0.1012,0.3254,0.0,8.50,31.33,0.5744,S6COND
1,IJS,iShares S&P Small-Cap 600 Value ETF,S&P Small Cap 600 Value Index,Value Small Cap,12.67,-1.57,-1.67,0.2048,0.1368,0.2733,0.0,9.54,26.96,0.7016,SMLV
2,SLYV,SPDR S&P 600 Small CapValue ETF,S&P Small Cap 600 Value Index,Value Small Cap,12.67,-1.57,-1.67,0.2048,0.1368,0.2733,0.0,9.54,26.96,0.7016,SMLV
3,VIOV,Vanguard S&P Small-Cap 600 Value ETF,S&P Small Cap 600 Value Index,Value Small Cap,12.67,-1.57,-1.67,0.2048,0.1368,0.2733,0.0,9.54,26.96,0.7016,SMLV
4,SPSM,SPDR Portfolio S&P 600 Small Cap ETF,S&P Small Cap 600 Index,Equity Small Cap,16.28,-1.69,-1.78,0.2009,0.1311,0.2706,0.0,11.13,31.86,0.6329,SML
5,IJR,iShares Core S&P Small-Cap ETF,S&P Small Cap 600 Index,Equity Small Cap,16.28,-1.69,-1.78,0.2009,0.1311,0.2706,0.0,11.13,31.86,0.6329,SML
6,FYX,First Trust Small Cap Core AlphaDEX Fund,S&P Small Cap 600 Index,Core Small Cap,16.28,-1.69,-1.78,0.2009,0.1311,0.2706,0.0,11.13,31.86,0.6329,SML
7,VIOO,Vanguard S&P Small-Cap 600 ETF,S&P Small Cap 600 Index,Equity Small Cap,16.28,-1.69,-1.78,0.2009,0.1311,0.2706,0.0,11.13,31.86,0.6329,SML
8,VIOG,Vanguard S&P Small-Cap 600 Growth ETF,S&P Small Cap 600 Growth Index,Growth Small Cap,24.51,-1.31,-1.29,0.1760,0.1043,0.2475,0.0,13.48,45.97,0.6426,SMLG
9,IJT,iShares S&P Small-Cap 600 Growth ETF,S&P Small Cap 600 Growth Index,Growth Small Cap,24.51,-1.31,-1.29,0.1760,0.1043,0.2475,0.0,13.48,45.97,0.6426,SMLG


In [6]:
symbols = list(selected_equity_etfs.TICKER)
remove_ticker_list = [pd.read_csv(r'C:\Code\Python\etradebot\data\remove_tickers.csv').squeeze()]
symbols = [ticker for ticker in symbols if ticker not in remove_ticker_list]

In [7]:
# root_dir = os.path.dirname(os.path.dirname(__file__))  # Get the root directory of the project
# bounds_path = os.path.join(root_dir, 'data', 'bounds.csv')
bounds_df = pd.read_csv(r'C:\Code\Python\etradebot\data\bounds.csv')
bounds = []

for ticker in symbols:
    if ticker in bounds_df['TICKER'].values:
        min_bound = bounds_df.loc[bounds_df['TICKER'] == ticker, 'MIN'].values[0]
        max_bound = bounds_df.loc[bounds_df['TICKER'] == ticker, 'MAX'].values[0]
        bounds.append((min_bound, max_bound))
    else:
        bounds.append((-0.30, 1.00))  # default bounds if ticker not in bounds.csv

In [8]:
api = DataAPI(
    symbols,
    num_years=100,  # years
    period='max',  # '1d', '5d', '7d', '60d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max'
    api_source='bloomberg',  # yahoo or bloomberg
)

In [9]:
model = Model(
    symbols,
    bounds,
    gamma,
    min_weight,
    margin_rate,
    long_weight,
    short_weight,
    frequency
)

In [10]:
historical_prices = api.get_historical_prices()
historical_prices

,SPYV,VOOV,IWF,SCHG,TMFC,VONG,MGK,PSCD,IJS,SLYV,...,FYX,VIOO,VIOG,IJT,SLYG,FDIS,VUG,ILCG,IUSV,IVE
date,,,,,,,,,,,,,,,,,,,,,
2018-01-30,27.6631,100.5004,136.7194,36.2343,19.4240,34.8636,114.1971,57.7283,71.5614,57.3132,...,59.0109,67.3316,71.9280,83.9188,55.5860,40.1982,143.0306,32.8825,50.4327,104.4580
2018-01-31,27.6371,100.4127,136.9381,36.3206,19.4631,34.9488,114.5517,57.6440,71.0455,56.9515,...,58.6617,66.9754,71.7055,83.5443,55.3393,40.0848,143.2785,33.0605,50.3539,104.3351
2018-02-01,27.6544,100.5618,136.4817,36.2487,19.4631,34.8116,113.9767,57.4200,71.2555,57.0855,...,58.7655,67.2895,71.8523,84.0184,55.6447,39.6973,142.7255,32.9019,50.4239,104.4492
2018-02-02,27.0228,98.3246,133.7342,35.4961,19.0136,34.1231,111.6764,55.8434,69.7762,55.8752,...,57.4348,65.8315,70.4600,82.4112,54.6016,39.2247,139.8456,32.3000,49.2600,102.0173
2018-02-05,25.9065,94.2186,128.4959,34.1059,18.3199,32.7697,107.3922,54.8405,67.1830,53.7672,...,55.3681,63.3563,67.7182,79.1352,52.5224,37.9865,134.2670,30.9998,47.1860,97.7768
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-13,45.8400,166.0600,300.2900,81.9300,44.1900,77.0860,257.4800,98.2314,99.2300,79.9574,...,88.4714,96.4900,103.3300,120.1400,80.1131,77.3516,307.5200,67.1311,83.2200,171.6400
2023-12-14,46.1684,167.2600,299.6400,81.6800,44.0500,76.9263,256.2600,102.4860,102.4200,82.5277,...,90.7617,99.2100,105.9900,123.1200,82.1165,78.6790,307.0700,67.1000,83.9700,172.9100
2023-12-15,46.0589,166.7600,300.6400,81.9400,44.2200,77.0760,257.1700,101.1473,101.2800,81.6643,...,90.0403,98.3500,105.4000,122.1700,81.4786,78.6800,307.9100,67.2000,83.6900,172.3900


In [11]:
risk_free_rate = api.get_risk_free_rate(prints=True)

------------------------------------
Risk Free Rate: 3.91%


In [12]:
market_caps = api.get_market_caps(prints=True)


Market Cap ($Millions):
VUG     $105,511.44
IWF      $81,973.60
IJR      $76,438.33
IVE      $27,385.93
SCHG     $22,682.14
SPYV     $19,923.42
VONG     $17,029.55
MGK      $16,508.91
IUSV     $15,540.10
SPSM      $9,709.57
IJS       $7,571.94
IJT       $5,553.51
VOOV      $4,066.38
SLYV      $3,977.27
SLYG      $3,018.17
VIOO      $2,497.49
ILCG      $1,918.19
FDIS      $1,410.94
VIOV      $1,383.87
FYX         $872.00
VIOG        $659.98
TMFC        $637.31
PSCD         $35.12
Name: CUR_MKT_CAP, dtype: object


In [13]:
vif_symbols = vif.vif_filter(historical_prices, market_caps, threshold=50, prints=True, omit_method='market_cap')


VIF Tickers:
TICKER
PSCD     5.53
IVE      7.38
FDIS    11.68
FYX     33.10
IJR     39.25
TMFC    40.35
VUG     47.72
Name: VIF Factor, dtype: float64


In [14]:
investor_views, confidences = views.investor_views_confidences(selected_equity_etfs, vif_symbols, prints=True)


Investor Views:
TICKER
PSCD    21.32%
FYX     20.09%
IJR     20.09%
FDIS    -0.42%
VUG     -2.66%
IVE     -3.13%
TMFC    -3.38%
Name: Investor Views, dtype: object

View Confidences:
TICKER
FYX     63.29%
IJR     63.29%
PSCD    57.44%
TMFC    46.96%
VUG     44.24%
IVE     42.84%
FDIS    16.53%
Name: Confidences, dtype: object


In [15]:
market_symbol, market_name, market_prices = api.get_market_prices(historical_prices, vif_symbols, prints=True)


Market Symbol: SPY US Equity
Market Name: SPDR S&P 500 ETF Trust
Market Prices:
date
2018-01-30    $254.82
2018-01-31    $254.95
2018-02-01    $254.66
2018-02-02    $249.11
2018-02-05    $238.69
               ...   
2023-12-13    $468.60
2023-12-14    $470.10
2023-12-15    $469.33
2023-12-18    $471.97
2023-12-19    $474.84
Name: SPDR S&P 500 ETF Trust, Length: 1483, dtype: object


In [16]:
market_implied_risk_aversion = model.market_implied_risk_aversion(market_prices, risk_free_rate, prints=True)


Market Implied Risk Aversion:
Delta: 2.09


In [17]:
covariance_matrix = model.calculate_covariance_matrix(historical_prices, vif_symbols, prints=True)


Covariance Matrix:
        FDIS     FYX     IJR     IVE    PSCD    TMFC     VUG
FDIS  0.0646  0.0579  0.0539  0.0418  0.0646  0.0553  0.0581
FYX   0.0579  0.0786  0.0722  0.0506  0.0803  0.0489  0.0523
IJR   0.0539  0.0722  0.0687  0.0476  0.0762  0.0457  0.0489
IVE   0.0418  0.0506  0.0476  0.0415  0.0491  0.0393  0.0413
PSCD  0.0646  0.0803  0.0762  0.0491  0.1110  0.0486  0.0528
TMFC  0.0553  0.0489  0.0457  0.0393  0.0486  0.0568  0.0578
VUG   0.0581  0.0523  0.0489  0.0413  0.0528  0.0578  0.0607


In [18]:
posterior_covariance_matrix, posterior_expected_returns = model.calculate_black_litterman(
    covariance_matrix,
    market_prices, 
    risk_free_rate, 
    market_caps, 
    vif_symbols, 
    investor_views,
    confidences, 
    prints=True
)


Market Implied Risk Aversion:
Delta: 2.09

Prior Expected Returns:
PSCD    16.64%
FYX     16.32%
IJR     15.61%
FDIS    15.30%
VUG     15.17%
TMFC    14.57%
IVE     13.02%
dtype: object

Posterior Expected Returns:
PSCD    17.54%
FYX     16.32%
IJR     15.61%
FDIS     8.32%
IVE      8.30%
VUG      5.93%
TMFC     5.23%
dtype: object


In [19]:
min_volatility, min_weights, min_results = model.minimum_risk_portfolio(
    posterior_expected_returns, posterior_covariance_matrix, risk_free_rate, prints=True
)

TypeError: test_bounds must be a pair (lower bound, upper bound) OR a collection of bounds for each asset

In [ ]:
max_volatility, max_weights, max_results = model.maximum_risk_portfolio(
    posterior_expected_returns, posterior_covariance_matrix, risk_free_rate, prints=True
)

In [ ]:
max_sharpe_weights, max_sharpe_results = model.maximum_sharpe_portfolio(
    posterior_expected_returns, posterior_covariance_matrix, risk_free_rate, prints=True
)

In [ ]:
portfolios, results = model.efficient_frontier_portfolios(
    posterior_expected_returns, posterior_covariance_matrix, risk_free_rate, prints=False
)
portfolios

In [ ]:
results

In [ ]:
model.plot_efficient_frontier(posterior_expected_returns, results, covariance_matrix, figsize=(12, 6))

In [ ]:
# max_sharpe_weights.to_csv('new_portfolio.csv')

In [ ]:
max_sharpe_weights